In [1]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름
# 스크립트가 실행되는 위치 기준으로 CSV 파일 경로를 지정하세요.
# 예: "업종_100m이내_버스정류장_목록.csv"
csv_file_name = "../data_csv/11_12/업종_100m이내_버스정류장_목록.csv"

# 생성할 SQL 파일 이름
sql_file_name = "insert_business_near_stops.sql"

# 생성할 테이블 이름 (따옴표 없음)
table_name = "business_near_stops"
# ----------------

def clean_value_mysql(value):
    """
    Python 데이터를 SQL 문자열로 안전하게 변환합니다.
    - None 또는 NaN -> 'NULL'
    - 문자열 -> 작은따옴표로 감싸고, 내부의 작은따옴표/역슬래시 이스케이프
    """
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        # MySQL은 '를 ''로, \를 \\로 이스케이프합니다.
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    elif isinstance(value, (int, float)):
        return str(value)
    else:
        # 기타 타입은 일단 문자열로 변환 시도
        cleaned_value = str(value).replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"

# 1. CSV 파일 읽기 (한글 인코딩 처리)
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig')
except FileNotFoundError:
    print(f"오류: '{csv_file_name}' 파일을 찾을 수 없습니다.")
    print("스크립트와 CSV 파일이 올바른 위치에 있는지 확인하세요.")
    df = pd.DataFrame() # 오류 시 빈 데이터프레임
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame()

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    
    # 2. 'Unnamed: 0' 컬럼명을 'original_index'로 변경 (SQL에서 사용 가능하게)
    if "Unnamed: 0" in df.columns:
        df = df.rename(columns={"Unnamed: 0": "original_index"})
        print("'Unnamed: 0' 컬럼을 'original_index'로 변경했습니다.")

    # 3. CREATE TABLE 쿼리 생성
    # 컬럼명 리스트 (따옴표 없음)
    columns = [col for col in df.columns]
    column_sql = ", ".join(columns)
    
    # 데이터 타입에 맞춰 CREATE TABLE 문 작성
    create_table_sql = f"""
-- 업종 100m 이내 버스정류장 목록 테이블 생성 (MySQL용, 따옴표 없음)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    original_index INT,
    업종 VARCHAR(255) CHARACTER SET utf8mb4,
    업소명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4,
    연락처 VARCHAR(50),
    위도_업종 DOUBLE,
    경도_업종 DOUBLE,
    index_정류장 INT,
    노선번호 VARCHAR(100),
    노선 DOUBLE,
    정류장순서 DOUBLE,
    정류장 INT,
    정류장명 VARCHAR(255) CHARACTER SET utf8mb4,
    경도_정류장 DOUBLE,
    위도_정류장 DOUBLE,
    데이터기준일 DATE
);
\n"""

    # 4. SQL 파일 쓰기 시작
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:
            # 파일 상단에 데이터베이스 선택 주석 추가
            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            
            # CREATE TABLE 문 쓰기
            f.write(create_table_sql)
            f.write("\n")
            
            values_batch = []
            
            # INSERT 문 시작 부분 (따옴표 없음)
            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"
            
            # 5. DataFrame을 행별로 순회하며 VALUES 생성 (명시적 방식)
            for row in df.itertuples(index=False):
                # 모든 컬럼의 데이터를 명시적으로 변환
                cleaned_row = [
                    clean_value_mysql(row.original_index),
                    clean_value_mysql(row.업종),
                    clean_value_mysql(row.업소명),
                    clean_value_mysql(row.주소),
                    clean_value_mysql(row.연락처),
                    clean_value_mysql(row.위도_업종),
                    clean_value_mysql(row.경도_업종),
                    clean_value_mysql(row.index_정류장),
                    clean_value_mysql(row.노선번호),
                    clean_value_mysql(row.노선),
                    clean_value_mysql(row.정류장순서),
                    clean_value_mysql(row.정류장),
                    clean_value_mysql(row.정류장명),
                    clean_value_mysql(row.경도_정류장),
                    clean_value_mysql(row.위도_정류장),
                    clean_value_mysql(row.데이터기준일)
                ]
                values_batch.append(f"({', '.join(cleaned_row)})")
            
            # 6. INSERT 쿼리 파일에 쓰기
            if values_batch:
                # 모든 데이터를 하나의 INSERT 문으로 합쳐서 씀
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")

'../data_csv/11_12/업종_100m이내_버스정류장_목록.csv' 파일 읽기 성공. 22개 행 발견.
'Unnamed: 0' 컬럼을 'original_index'로 변경했습니다.
'insert_business_near_stops.sql' 파일 생성이 완료되었습니다.
총 22개의 행이 파일에 저장되었습니다.
